# Convolution Neural Network

CNN은 크게 Feature Extractor와 Classifier, 2가지 부분으로 구분된다.

Input으로 부터 Feature를 추출하는 역할은 Convolution, Pooling Layer로 수행하며,

추출된 Feature를 Flatten 하여, Dense Layer를 사용한 Classifier가 Input을 분류하게 된다.

## Lenet

예를 들어, 다음과 같은 구조를 갖는 CNN 모델을 생각할 수 있다.

X => Conv2D => AvgPooling2D => Conv2D => AvgPooling2D => Conv2D => Flatten (여기까지 Feature Extractor) => Dense => Dense => Softmax (Classifier)

## 실습

### Shapes in the Feature Extractors

In [11]:
import torch

N, I_c, I_h, I_w = 32, 3, 28, 28
n_conv_filter = 5
kernel_size = 3
pool_size, pool_strides = 2, 2
batch_size = 32

x = torch.randn(N, I_c, I_h, I_w)

conv1 = torch.nn.Conv2d(I_c, n_conv_filter, kernel_size = kernel_size, padding = 1)
relu = torch.nn.ReLU()
conv1_pool = torch.nn.MaxPool2d(kernel_size = pool_size, stride = pool_strides)

conv2 = torch.nn.Conv2d(n_conv_filter, n_conv_filter, kernel_size = kernel_size, padding = 1)
conv2_pool = torch.nn.MaxPool2d(kernel_size = pool_size, stride = pool_strides)

flatten = torch.nn.Flatten()

print(f"Input : {x.detach().numpy().shape}")

x = conv1(x)
x = relu(x)
W, B = conv1.weight, conv1.bias
print(f"W/B : {W.detach().numpy().shape}/{B.detach().numpy().shape}")
print(f"After conv1 : {x.detach().numpy().shape}")
x = conv1_pool(x)
print(f"After conv pool1 : {x.detach().numpy().shape}")

x = conv2(x)
x = relu(x)
W, B = conv2.weight, conv2.bias
print(f"W/B : {W.detach().numpy().shape}/{B.detach().numpy().shape}")
print(f"After conv2 : {x.detach().numpy().shape}")
x = conv2_pool(x)
print(f"After conv pool2 : {x.detach().numpy().shape}")

x = flatten(x)
print(f"After flatten : {x.detach().numpy().shape}")

Input : (32, 3, 28, 28)
W/B : (5, 3, 3, 3)/(5,)
After conv1 : (32, 5, 28, 28)
After conv pool1 : (32, 5, 14, 14)
W/B : (5, 5, 3, 3)/(5,)
After conv2 : (32, 5, 14, 14)
After conv pool2 : (32, 5, 7, 7)
After flatten : (32, 245)


### Shapes in the Classifier

In [12]:
import torch

n_neurons = [50, 25, 10]

dense1 = torch.nn.Linear(245, n_neurons[0])
dense2 = torch.nn.Linear(n_neurons[0], n_neurons[1])
dense3 = torch.nn.Linear(n_neurons[1], n_neurons[2])
relu = torch.nn.ReLU()
softmax = torch.nn.Softmax(dim = 1)

print(f"Input feature : {x.detach().numpy().shape}")

x = dense1(x)
x = relu(x)
W, B = dense1.weight, dense1.bias
print(f"W/B : {W.detach().numpy().shape}/{B.detach().numpy().shape}")
print(f"After dense1 : {x.detach().numpy().shape}")

x = dense2(x)
x = relu(x)
W, B = dense2.weight, dense2.bias
print(f"W/B : {W.detach().numpy().shape}/{B.detach().numpy().shape}")
print(f"After dense2 : {x.detach().numpy().shape}")

x = dense3(x)
x = softmax(x)
W, B = dense3.weight, dense3.bias
print(f"W/B : {W.detach().numpy().shape}/{B.detach().numpy().shape}")
print(f"After dense3 : {x.detach().numpy().shape}")

Input feature : (32, 245)
W/B : (50, 245)/(50,)
After dense1 : (32, 50)
W/B : (25, 50)/(25,)
After dense2 : (32, 25)
W/B : (10, 25)/(10,)
After dense3 : (32, 10)


### Implementation with Sequential Method

In [17]:
import torch

N, I_c, I_h, I_w = 4, 3, 28, 28
n_conv_neurons = [10, 20, 30]
n_dense_neurons = [50, 30, 10]
kernel_size = 3
padding = 1
pool_size, pool_strides = 2, 2

x = torch.randn(N, I_c, I_h, I_w)

model = torch.nn.Sequential()
model.add_module('conv1', torch.nn.Conv2d(I_c, n_conv_neurons[0], kernel_size = kernel_size, padding = padding))
model.add_module('relu1', torch.nn.ReLU())
model.add_module('MaxPool1', torch.nn.MaxPool2d(kernel_size = pool_size, stride = pool_strides))
model.add_module('flatten', torch.nn.Flatten())

model.add_module('dense1', torch.nn.Linear(1960, n_dense_neurons[0]))
model.add_module('relu2', torch.nn.ReLU())
model.add_module('dense2', torch.nn.Linear(n_dense_neurons[0], n_dense_neurons[1]))
model.add_module('relu3', torch.nn.ReLU())
model.add_module('dense3', torch.nn.Linear(n_dense_neurons[1], n_dense_neurons[2]))
model.add_module('softmax', torch.nn.Softmax(dim = 1))

### Implementation with Model Sub-classing

In [22]:
import torch

class TestCNN(torch.nn.Module):
    def __init__(self):
        super(TestCNN, self).__init__()
        
        self.conv1 = torch.nn.Conv2d(I_c, n_conv_neurons[0], kernel_size = kernel_size, padding = padding)
        self.relu = torch.nn.ReLU()
        self.maxpool = torch.nn.MaxPool2d(kernel_size = pool_size, stride = pool_strides)
        self.conv2 = torch.nn.Conv2d(n_conv_neurons[0], n_conv_neurons[1], kernel_size = kernel_size, padding = padding)
        self.conv3 = torch.nn.Conv2d(n_conv_neurons[1], n_conv_neurons[2], kernel_size = kernel_size, padding = padding)

        
        self.flatten = torch.nn.Flatten()
        
        self.dense1 = torch.nn.Linear(270, n_dense_neurons[0])
        self.dense2 = torch.nn.Linear(n_dense_neurons[0], n_dense_neurons[1])
        self.dense3 = torch.nn.Linear(n_dense_neurons[1], n_dense_neurons[2])
        
        self.softmax = torch.nn.Softmax(dim = 1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.conv3(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.flatten(x)
        
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        x = self.relu(x)
        x = self.dense3(x)
        x = self.softmax(x)
        
N, I_c, I_h, I_w = 4, 3, 28, 28
n_conv_neurons = [10, 20, 30]
n_dense_neurons = [50, 30, 10]
kernel_size = 3
padding = 1
pool_size, pool_strides = 2, 2

x = torch.randn(N, I_c, I_h, I_w)
model = TestCNN()
y = model(x)

### Implementation with Sequential + layer sub-classing

In [30]:
import torch

class MyConv(torch.nn.Module):
    def __init__(self, in_channel, out_channel):
        super(MyConv, self).__init__()
        self.conv = torch.nn.Conv2d(in_channel, out_channel, kernel_size = kernel_size, padding = padding)
        self.relu = torch.nn.ReLU()
        self.pool = torch.nn.MaxPool2d(kernel_size = pool_size, stride = pool_strides)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.pool(x)
        return x
    
model = torch.nn.Sequential()
model.add_module('conv1', MyConv(I_c, n_conv_neurons[0]))
model.add_module('conv2', MyConv(n_conv_neurons[0], n_conv_neurons[1]))
model.add_module('conv3', MyConv(n_conv_neurons[1], n_conv_neurons[2]))
model.add_module('flatten', torch.nn.Flatten())

model.add_module('dense1', torch.nn.Linear(270, n_dense_neurons[0]))
model.add_module('relu2', torch.nn.ReLU())
model.add_module('dense2', torch.nn.Linear(n_dense_neurons[0], n_dense_neurons[1]))
model.add_module('relu3', torch.nn.ReLU())
model.add_module('dense3', torch.nn.Linear(n_dense_neurons[1], n_dense_neurons[2]))
model.add_module('softmax', torch.nn.Softmax(dim = 1))

x = torch.randn(N, I_c, I_h, I_w)
y = model(x)

### Implementation with Model and Layer-Subclassing

In [32]:
import torch

class MyConv(torch.nn.Module):
    def __init__(self, in_channel, out_channel):
        super(MyConv, self).__init__()
        self.conv = torch.nn.Conv2d(in_channel, out_channel, kernel_size = kernel_size, padding = padding)
        self.relu = torch.nn.ReLU()
        self.pool = torch.nn.MaxPool2d(kernel_size = pool_size, stride = pool_strides)
        
        
    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.pool(x)
        return x
    
class TestCNN(torch.nn.Module):
    def __init__(self):
        super(TestCNN, self).__init__()
        
        self.conv1 = MyConv(I_c, n_conv_neurons[0])
        self.conv2 = MyConv(n_conv_neurons[0], n_conv_neurons[1])
        self.conv3 = MyConv(n_conv_neurons[1], n_conv_neurons[2])
        self.relu = torch.nn.ReLU()
        self.flatten = torch.nn.Flatten()
        
        self.dense1 = torch.nn.Linear(270, n_dense_neurons[0])
        self.dense2 = torch.nn.Linear(n_dense_neurons[0], n_dense_neurons[1])
        self.dense3 = torch.nn.Linear(n_dense_neurons[1], n_dense_neurons[2])
        self.softmax = torch.nn.Softmax(dim = 1)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        x = self.relu(x)
        x = self.dense3(x)
        x = self.softmax(x)
        
        return x
    
x = torch.randn(N, I_c, I_h, I_w)
model = TestCNN()
y = model(x)

### LeNet with Model Sub-classing

In [35]:
import torch

class LeNet(torch.nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        
        self.tanh = torch.nn.Tanh()
        self.softmax = torch.nn.Softmax(dim = 1)
        self.avgpool = torch.nn.AvgPool2d(kernel_size = 2)
        
        self.conv1 = torch.nn.Conv2d(1, 6, kernel_size = 5)
        self.conv2 = torch.nn.Conv2d(6, 16, kernel_size = 5)
        self.conv3 = torch.nn.Conv2d(16, 120, kernel_size = 5)
        
        self.flatten = torch.nn.Flatten()
        
        self.dense1 = torch.nn.Linear(120, 84)
        self.dense2 = torch.nn.Linear(84, 10)
        
    def forward(self, x):
        print("x: {}".format(x. shape))
        x = self.conv1(x)
        x = self.tanh(x)
        print("x: {}".format(x.shape))
        x = self.avgpool(x)
        print("x: {}".format(x.shape))
        x = self.conv2(x)
        x = self.tanh(x)
        print("x: {}".format(x.shape))
        x = self.avgpool(x)
        print("x: {}".format(x.shape))
        x = self.conv3(x)
        x = self.tanh(x)
        print("x: {}".format(x.shape))
        x = self. flatten(x)
        print("x: {}".format(x.shape))
        x = self.dense1(x) 
        x = self.tanh(x)
        print("x: {}".format(x.shape))
        x = self.dense2(x)
        x = self.softmax(x)
        print("x: {}".format(x.shape))
        return x
    
model = LeNet()

### LeNet with Hybrid Method

In [40]:
import torch

class ConvLayer(torch.nn.Module):
    def __init__(self, in_channel, out_channel, pool):
        super(ConvLayer, self).__init__()
        self.ispool = pool
        self.conv = torch.nn.Conv2d(in_channel, out_channel, kernel_size = 5)
        self.tanh = torch.nn.Tanh()
        if pool:
            self.pool = torch.nn.AvgPool2d(kernel_size = 2)
        
        
    def forward(self, x):
        x = self.conv(x)
        x = self.tanh(x)
        if self.ispool:
            x = self.pool(x)
        return x
    
class LeNet(torch.nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = ConvLayer(1, 6, True)
        self.conv2 = ConvLayer(6, 16, True)
        self.conv3 = ConvLayer(16, 120, False)
        self.flatten = torch.nn.Flatten()
        self.dense1 = torch.nn.Linear(120, 84)
        self.dense2 = torch.nn.Linear(84, 10)
        self.softmax = torch.nn.Softmax(dim = 1)
        self.tanh = torch.nn.Tanh()
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.tanh(x)
        x = self.dense2(x)
        x = self.softmax(x)
        return x
    
model = LeNet()

x = torch.randn(32, 1, 32, 32)
predictions = model(x)

### Forward Propagation of LeNet

In [52]:
import torch
from torchvision import datasets, transforms

class ConvLayer(torch.nn.Module):
    def __init__(self, in_channel, out_channel, pool):
        super(ConvLayer, self).__init__()
        self.ispool = pool
        self.conv = torch.nn.Conv2d(in_channel, out_channel, kernel_size = 5)
        self.tanh = torch.nn.Tanh()
        if pool:
            self.pool = torch.nn.AvgPool2d(kernel_size = 2)
        
        
    def forward(self, x):
        x = self.conv(x)
        x = self.tanh(x)
        if self.ispool:
            x = self.pool(x)
        return x
    
class LeNet(torch.nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = ConvLayer(1, 6, True)
        self.conv2 = ConvLayer(6, 16, True)
        self.conv3 = ConvLayer(16, 120, False)
        self.flatten = torch.nn.Flatten()
        self.dense1 = torch.nn.Linear(120, 84)
        self.dense2 = torch.nn.Linear(84, 10)
        self.softmax = torch.nn.Softmax(dim = 1)
        self.tanh = torch.nn.Tanh()
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.tanh(x)
        x = self.dense2(x)
        x = self.softmax(x)
        return x

### Dataset Preparation ###
transforms = transforms.Compose([transforms.Resize((32, 32)),
                                 transforms.ToTensor()])
dataset = datasets.MNIST(root='mnist_data', 
                               train=True, 
                               transform=transforms,
                               download=True)
dataloader = torch.utils.data.DataLoader(dataset=dataset, 
                                         batch_size=32, 
                                         shuffle=True)

### Foward Propagation ###
model = LeNet()

loss_function = torch.nn.CrossEntropyLoss()
for images, labels in dataloader:
    predictions = model(images)
    loss = loss_function(predictions, labels)
    print(loss.detach().numpy())
    break

2.3007827
